In [1]:
#Checking if GPU is running or not
!nvidia-smi

Sat Sep 23 22:27:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.83       Driver Version: 528.83       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8     5W /  30W |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install datasets

In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [4]:
# !pip install transformers

In [5]:
# !pip install tensorflow

In [6]:
!pip install transformers[sentencepiece]

In [8]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

# english to hindi model

In [79]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

# loading datasets

In [10]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

Repo card metadata block was not found. Setting CardData to empty.


In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [12]:
raw_datasets['train'][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

# data preprocessing

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

N:\deep learning project\RNN\english-to-hindi-translator\englist_to_hindi\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


N:\deep learning project\RNN\english-to-hindi-translator\englist_to_hindi\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [15]:
tokenizer("my name is nitin budhlakoti")

{'input_ids': [155, 300, 23, 6532, 1659, 568, 23327, 2326, 3813, 6915, 8494, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:

with tokenizer.as_target_tokenizer():
    print(tokenizer(["मेरा नाम नितिन है"]))

{'input_ids': [[500, 179, 21441, 5138, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}


N:\deep learning project\RNN\english-to-hindi-translator\englist_to_hindi\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [25]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # print(inputs)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [27]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [28]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6d4e7408-d86c-41f1-afec-a22f2582e61e)')' thrown while requesting HEAD https://huggingface.co/Helsinki-NLP/opus-mt-en-hi/resolve/main/config.json


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [29]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [31]:
data_collator

DataCollatorForSeq2Seq(tokenizer=MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-hi', vocab_size=61950, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True), model=<transformers.models.marian.modeling_tf_marian.TFMarianMTModel object at 0x0000029C375B9090>, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='tf')

In [32]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [34]:
generation_data_collator

DataCollatorForSeq2Seq(tokenizer=MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-hi', vocab_size=61950, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True), model=<transformers.models.marian.modeling_tf_marian.TFMarianMTModel object at 0x0000029C375B9090>, padding=True, max_length=None, pad_to_multiple_of=128, label_pad_token_id=-100, return_tensors='tf')

In [42]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [43]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [44]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [45]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [47]:
# model_training
model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

Epoch 1/10
156/156 [==============================] - 502s 3s/step - loss: 3.5568 - val_loss: 3.8759
Epoch 2/10
156/156 [==============================] - 470s 3s/step - loss: 3.2054 - val_loss: 3.8324
Epoch 3/10
156/156 [==============================] - 465s 3s/step - loss: 2.9382 - val_loss: 3.8164
Epoch 4/10
156/156 [==============================] - 463s 3s/step - loss: 2.6962 - val_loss: 3.8033
Epoch 5/10
156/156 [==============================] - 467s 3s/step - loss: 2.4993 - val_loss: 3.8035
Epoch 6/10
156/156 [==============================] - 463s 3s/step - loss: 2.3184 - val_loss: 3.8186
Epoch 7/10
156/156 [==============================] - 465s 3s/step - loss: 2.1592 - val_loss: 3.8284
Epoch 8/10
156/156 [==============================] - 474s 3s/step - loss: 2.0032 - val_loss: 3.8465
Epoch 9/10
156/156 [==============================] - 475s 3s/step - loss: 1.8598 - val_loss: 3.8675
Epoch 10/10
156/156 [==============================] - 545s 3s/step - loss: 1.7319 - val_lo

In [48]:
# saving our model

In [49]:
model.save_pretrained("tf_model/")

# model testing

In [50]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 274462cb-8a14-4197-8b83-85d7f79280e1)')' thrown while requesting HEAD https://huggingface.co/Helsinki-NLP/opus-mt-en-hi/resolve/main/tokenizer_config.json
N:\deep learning project\RNN\english-to-hindi-translator\englist_to_hindi\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [67]:
input_text  = "The constitute of our country guarantees freedom of speech."

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   217   534     6  2474   313    15 13237     5    25  4452    15
   6921     5    40     0]], shape=(1, 16), dtype=int32)


In [68]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

हमारे देश के सदस्य होने की गारंटी है कि बोलने की स्वतंत्रता है।


In [73]:
def english_to_hindi(input_text):
    tokenized=tokenizer([input_text],return_tensors='np')
    out=model.generate(**tokenized,max_length=128)
    print(out)
    with tokenizer.as_target_tokenizer():
        print(tokenizer.decode(out[0],skip_special_tokens=True))
        

In [76]:
docs="""The advancement of technology has transformed the way governments operate. In this digital age, governments are increasingly relying on data-driven decision-making and online services to meet the needs of their citizens. From e-governance initiatives to smart city projects, technology is at the forefront of modern governance.

One of the critical aspects of this transformation is the translation of government documents and information into multiple languages to ensure accessibility and inclusivity. English to Hindi translation plays a vital role in India, where Hindi is one of the official languages and widely spoken.

Government agencies need robust translation tools to communicate effectively with the Hindi-speaking population. Whether its legal documents, healthcare information, or policy announcements, accurate translation is essential to convey the intended message.

However, translation is not just about converting words from one language to another. It involves capturing the nuances of language, understanding cultural sensitivities, and ensuring the accuracy of the content. This is especially crucial in government communications, where precision and clarity are paramount.

As governments continue to expand their digital footprint, the demand for reliable translation services will only increase. Its not just a matter of convenience; its a matter of equitable access to government information and services.

In conclusion, the role of English to Hindi translation in government operations cannot be overstated. It bridges language gaps, promotes transparency, and empowers citizens to engage with their government. As technology continues to evolve, so too will the tools and solutions available for seamless translation, making government services more accessible and efficient for all.
"""

In [78]:
english_to_hindi(docs)

tf.Tensor(
[[61949 34882    15  4991    37   108   438    24  7001    78     5     2
    263   163    12 11501    37  7284    15     5    40    89   201     2
  11501    37    63 21324    15  5483   966    18   444    91     6    39
   4977    69 26901   314  5048  1231  4871 23862    33 12060   246    12
   5584    78     5    40    60  6073  1463  1266     6  6819  5249    28
     40    89  3305     6   207    11  3527     6  7001     6    38  5011
   5249     5     2  1231  1265  1152    11  1447    15 10359     2  1231
    510   204    25    60   181    47 22320  1152    11  3321    29    78
    209   168    40    89  1426    11     2  3451   169  3527     6    99
     18  8634 29862   260 16888   260 16888   260 16888   246    11  3321
     91    15  5483    29     5     2  1231     0]], shape=(1, 128), dtype=int32)
प्रौद्योगिकी की प्रगति ने उस प्रकार का विकास किया है, जिस तरह से सरकारों ने कार्रवाई की है। इस समय, सरकारों ने अपने नागरिकों की आवश्यकताओं को पूरा करने के लिए डेटा-चाल